In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd '/content/drive/MyDrive/Data'

/content/drive/MyDrive/Data


In [2]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
import math
import numpy as np
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.metrics import confusion_matrix, classification_report
from collections import defaultdict
from textwrap import wrap

In [3]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB, GaussianNB
from sklearn.linear_model import SGDClassifier, LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold, KFold
from sklearn.svm import SVC, LinearSVC
from sklearn import metrics, utils
import sklearn
from xgboost import XGBClassifier
import xgboost as xgb
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
os.getcwd()

'/content/drive/MyDrive/Data'

In [5]:
df = pd.read_csv('isarcasmeval_train.csv')

In [8]:
print(df.head())

   Unnamed: 0                                              tweet  sarcastic  \
0           0  The only thing I got from college is a caffein...          1   
1           1  I love it when professors draw a big question ...          1   
2           2  Remember the hundred emails from companies whe...          1   
3           3  Today my pop-pop told me I was not “forced” to...          1   
4           4  @VolphanCarol @littlewhitty @mysticalmanatee I...          1   

                                            rephrase  sarcasm  irony  satire  \
0  College is really difficult, expensive, tiring...      0.0    1.0     0.0   
1  I do not like when professors don’t write out ...      1.0    0.0     0.0   
2  I, at the bare minimum, wish companies actuall...      0.0    1.0     0.0   
3  Today my pop-pop told me I was not "forced" to...      1.0    0.0     0.0   
4  I would say Ted Cruz is an asshole and doesn’t...      1.0    0.0     0.0   

   understatement  overstatement  rhetorical

In [6]:
df = df[['tweet', 'rephrase']].dropna()
print(df)

                                                 tweet  \
0    The only thing I got from college is a caffein...   
1    I love it when professors draw a big question ...   
2    Remember the hundred emails from companies whe...   
3    Today my pop-pop told me I was not “forced” to...   
4    @VolphanCarol @littlewhitty @mysticalmanatee I...   
..                                                 ...   
862             yo @claires do yall do hysterectomies?   
863  @JacobWohlReport Do I need to aquire a wife be...   
864  I get a lot of boy who cried wolf vibes from t...   
865  Update: holding hands with your mom and walkin...   
866  I might be rubbish at driving, and have a less...   

                                              rephrase  
0    College is really difficult, expensive, tiring...  
1    I do not like when professors don’t write out ...  
2    I, at the bare minimum, wish companies actuall...  
3    Today my pop-pop told me I was not "forced" to...  
4    I would say T

In [7]:
df['text'] = df['tweet'] + '[SEP]' + df['rephrase']
df['text_reverse'] = df['rephrase'] + '[SEP]' + df['tweet']
df['label'] = 0
df_merged_0 = df[['text', 'label']]
print(df_merged_0)

                                                  text  label
0    The only thing I got from college is a caffein...      0
1    I love it when professors draw a big question ...      0
2    Remember the hundred emails from companies whe...      0
3    Today my pop-pop told me I was not “forced” to...      0
4    @VolphanCarol @littlewhitty @mysticalmanatee I...      0
..                                                 ...    ...
862  yo @claires do yall do hysterectomies?[SEP]Cla...      0
863  @JacobWohlReport Do I need to aquire a wife be...      0
864  I get a lot of boy who cried wolf vibes from t...      0
865  Update: holding hands with your mom and walkin...      0
866  I might be rubbish at driving, and have a less...      0

[867 rows x 2 columns]


In [8]:
df_merged_1 = df[['text_reverse']]
df_merged_1['label'] = 1
df_merged_1.rename(columns={'text_reverse': 'text'}, inplace=True)
print(df_merged_1)

                                                  text  label
0    College is really difficult, expensive, tiring...      1
1    I do not like when professors don’t write out ...      1
2    I, at the bare minimum, wish companies actuall...      1
3    Today my pop-pop told me I was not "forced" to...      1
4    I would say Ted Cruz is an asshole and doesn’t...      1
..                                                 ...    ...
862  Claires, you should not do full hysterectomies...      1
863  A lot of people don't have a wife.[SEP]@JacobW...      1
864  The red cross is always needy. [SEP]I get a lo...      1
865  Holding hands with your parent while walking a...      1
866  Unfortunately, one of my few cookery skills do...      1

[867 rows x 2 columns]


<ipython-input-8-3aa63fe8ab36>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_1['label'] = 1
<ipython-input-8-3aa63fe8ab36>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged_1.rename(columns={'text_reverse': 'text'}, inplace=True)


In [9]:
df_merged = pd.concat([df_merged_0[:433], df_merged_1[433:]], ignore_index=True)
df_merged = df_merged.sample(frac=1).reset_index(drop=True) #shuffling
print(df_merged)

                                                  text  label
0    My dream is to be left alone.[SEP]I simply do ...      0
1    me and noah forgot that today is our anniversa...      0
2    We should have spent the money we wasted killi...      1
3    I really don't like falling up the stairs.[SEP...      1
4    My work email isn’t working so I can just log ...      0
..                                                 ...    ...
862  Just like to congratulate everyone on the Kop ...      0
863  "I got asked if I needed a kid's menu today......      1
864  Stranger Things is pretty good but season 1 is...      1
865  It's nice to see some heritage organisations a...      0
866  Been trying to get my boyfriends attention. Le...      0

[867 rows x 2 columns]


In [10]:
df_train, df_test = train_test_split(df_merged, test_size=0.2)

In [ ]:
import re
import emoji
import contractions

def expand_contractions(text):
    expanded_text = contractions.fix(text)
    return expanded_text

def preprocess_tweet(tweet):
  # Replace tagged users with @user
  tweet = re.sub(r'@[\w]+', '@user', tweet)

  # Replace multiple white spaces with a single space
  tweet = re.sub(r'\s+', ' ', tweet)

  # Remove more than two successive occurrences of any punctuation
  tweet = re.sub(r'(\s*([.,!?;:()<>{}[\]\-+=~_$%^&]\s){2,})', r'\1', tweet)

  # Replace emoji with emoji descriptor
  tweet = emoji.demojize(tweet)

  # Put a single white space around punctuations
  tweet = re.sub(r'([.,!?;:()<>{}[\]\-+=~_$%^&*])', r' \1 ', tweet)

  # expand contractions
  tweet = expand_contractions(tweet)

  return tweet

In [ ]:
!pip install emoji

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.5/397.5 kB 7.4 MB/s eta 0:00:00


In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.8/110.8 kB 7.9 MB/s eta 0:00:00


In [ ]:
df_train['text'] = df_train['text'].apply(lambda x: preprocess_tweet(x))
df_test['text'] = df_test['text'].apply(lambda x: preprocess_tweet(x))

print(df_train['text'])

tokens_train = df_train['text'].apply(lambda x: ' '.join([val for val in word_tokenize(x.lower()) if val not in stop_words])).values
tokens_test = df_test['text'].apply(lambda x: ' '.join([val for val in word_tokenize(x.lower()) if val not in stop_words])).values

tfidf_vectorizer = TfidfVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
xtrain_tfidf = tfidf_vectorizer.fit_transform(tokens_train)
xtest_tfidf= tfidf_vectorizer.transform(tokens_test)

print(xtrain_tfidf.shape, xtest_tfidf.shape)

739    Making 'theft of a dog' a new offense in Engli...
63     I would put actual top ranked games ,  not stu...
388    Keanu reeves is killing his role in ozark seas...
865    Wow it is almost like we do not nap every sing...
716    Do not keep up with the Jones’s  [ SEP ] Do no...
                             ...                        
819    Holding hands with your parent while walking a...
99     Ah yes ,  it really is wood - burning stoves i...
459    There is still plenty of space at the ground [...
732    I worked extremely hard tonight and am crouche...
79     When I wrote this tweet ,  COVID testing was r...
Name: text, Length: 693, dtype: object
(693, 1000) (174, 1000)


In [ ]:
print(tokens_train)

['@ user @ user @ user michael saylor efficient line questioning . feels like interrogation : face _ _ tears _ _ joy : : hollow _ red _ circle : [ sep ] questions basic interview worth watching .'
 "`` tonight saying '' [ sep ] ever peep fox see crazy tuck saying ? tonight , learning fauci actually funded creation covid lab solely responsible pandemic"
 "worried biden 's tax plan [ sep ] people worried biden ’ tax plan r ones get gifts easter"
 ...
 '@ user think would chance fury mean easy broken bodied old boy beat best world [ sep ] haye competitive fight fury 50 - 50 fight could go either way'
 'worried may used slur saying work spunk today [ sep ] spunk slur ?'
 'really love hurting foot , really painful ! [ sep ] hurt foot like 3 days ago , idea , hurts put pressure . love']


In [11]:
y_train = df_train['label'].values
y_test = df_test['label'].values
print(y_train.shape, y_test.shape)

(693,) (174,)


**Machine Learning Models**

In [ ]:
rf = RandomForestClassifier(class_weight='balanced_subsample')
grid_rf = GridSearchCV(rf, {'n_estimators':[300,700], 'max_depth':[7,11]})

xgb_classifier = XGBClassifier()
grid_xgb = GridSearchCV(xgb_classifier, {'n_estimators':[500,700],
                                         'max_depth':[7,11]})

lr = LogisticRegressionCV(max_iter=1000)

lr_model = lr.fit(xtrain_tfidf, y_train)
rf_model = grid_rf.fit(xtrain_tfidf, y_train)
xgb_model = grid_xgb.fit(xtrain_tfidf, y_train)

y_pred_lr = lr_model.predict(xtest_tfidf)
y_pred_rf = rf_model.predict(xtest_tfidf)
y_pred_xgb = xgb_model.predict(xtest_tfidf)

print(f'Accuracy RF: {round(np.mean(y_pred_rf==y_test), 3)}')
print(f'Accuracy LR: {round(np.mean(y_pred_lr==y_test), 3)}')
print(f'Accuracy XGB: {round(np.mean(y_pred_xgb==y_test), 3)}')

print(f'F1 score RF: {round(f1_score(y_test, y_pred_rf), 3)}')
print(f'F1 score LR: {round(f1_score(y_test, y_pred_lr), 3)}')
print(f'F1 score XGB: {round(f1_score(y_test, y_pred_xgb), 3)}')

Accuracy RF: 0.477
Accuracy LR: 0.494
Accuracy XGB: 0.517
F1 score RF: 0.492
F1 score LR: 0.488
F1 score XGB: 0.506


In [ ]:
from sklearn.svm import SVC

svm_classifier = SVC()  # You can choose different kernels (linear, rbf, etc.)

param_grid = {
    'C': [0.1, 1, 10],
    'gamma': [1, 0.1, 0.01],
    'kernel': ['linear', 'rbf']
}

# Perform Grid Search to find the best parameters
grid_search = GridSearchCV(svm_classifier, param_grid, cv=3, n_jobs=-1)
grid_search.fit(xtrain_tfidf, y_train)

# Get the best parameters found by Grid Search
best_params = grid_search.best_params_
print("Best Parameters:", best_params)

# Use the best estimator found by Grid Search
best_svm = grid_search.best_estimator_

# Predict the labels for the test set using the best SVM classifier
y_pred_svm = best_svm.predict(xtest_tfidf)

print(f'Accuracy SVM: {round(np.mean(y_pred_svm==y_test), 3)}')
print(f'F1 score SVM: {round(f1_score(y_test, y_pred_svm), 3)}')

Best Parameters: {'C': 0.1, 'gamma': 1, 'kernel': 'linear'}
Accuracy SVM: 0.494
F1 score SVM: 0.662


**BERT**

In [13]:
!pip install transformers

In [18]:
!pip install accelerate -U

In [12]:
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer
from transformers import BertForSequenceClassification
from sklearn.metrics import accuracy_score, f1_score, classification_report
from transformers import Trainer

class SarcasmDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

## Test Dataset
class SarcasmTestDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        return item
    def __len__(self):
        return len(self.encodings)

def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    f1 = f1_score(labels, pred)

    return {"accuracy": accuracy,"f1_score":f1}

def labels(x):
    if x == 0:
        return 0
    else:
        return 1

In [ ]:
!pip install transformers[torch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 5.1 MB/s eta 0:00:00


In [16]:
train_tweets = df_train['text'].values.tolist()
train_labels = df_train['label'].values.tolist()
test_tweets = df_test['text'].values.tolist()
test_labels = df_test['label'].values.tolist()

train_tweets, val_tweets, train_labels, val_labels = train_test_split(train_tweets, train_labels,
                                                                    test_size=0.2,random_state=42,stratify=train_labels)

model_name = 'detecting-Sarcasm'


tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', num_labels=2,
                                        loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
val_encodings = tokenizer(val_tweets, padding=True, truncation=True, max_length=512)
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
val_dataset = SarcasmDataset(val_encodings, val_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)


training_args = TrainingArguments(
    output_dir="output",
    evaluation_strategy="steps",
    eval_steps=500,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    seed=0,
    load_best_model_at_end=True,
)


model = BertForSequenceClassification.from_pretrained("bert-base-uncased",num_labels=2)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset = val_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


{'eval_loss': 0.6927714347839355,
 'eval_accuracy': 0.5179856115107914,
 'eval_f1_score': 0.54421768707483,
 'eval_runtime': 1.0794,
 'eval_samples_per_second': 128.77,
 'eval_steps_per_second': 16.675,
 'epoch': 3.0}

In [17]:
test_tweets = df_test['text'].values.tolist()
test_labels = df_test['label'].values.tolist()
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(np.sum(test_labels == results))
print(classification_report(test_labels, results))

94
              precision    recall  f1-score   support

           0       0.51      0.56      0.53        82
           1       0.57      0.52      0.55        92

    accuracy                           0.54       174
   macro avg       0.54      0.54      0.54       174
weighted avg       0.54      0.54      0.54       174



**RoBERTa**

In [14]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

train_tweets = df_train['text'].astype(str).values.tolist()
train_labels = df_train['label'].values.tolist()
test_tweets = df_test['text'].astype(str).values.tolist()
test_labels = df_test['label'].values.tolist()

model_name = 'detecting-Sarcasm'

MODEL = f"cardiffnlp/twitter-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2,
                                          loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
                      output_dir='./',
                      evaluation_strategy="steps",
                      num_train_epochs=3,
                      per_device_train_batch_size=8,
                      per_device_eval_batch_size=8,
                      warmup_steps=500, weight_decay=0.01,
                      logging_dir='./logs4', load_best_model_at_end=True)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=train_dataset,
                  eval_dataset=test_dataset,
                  compute_metrics=compute_metrics)

trainer.train()
trainer.evaluate()

#test_tweets = test['text'].values.tolist()
#test_labels = test['sarcastic'].values.tolist()
#test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

config.json:   0%|          | 0.00/747 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

KeyboardInterrupt: ignored

**XLNet**

In [15]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.4 MB/s eta 0:00:00


In [14]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, recall_score, precision_score
from sklearn.utils import shuffle
from transformers import XLNetTokenizer
from transformers import XLNetForSequenceClassification

train_tweets = df_train['text'].astype(str).values.tolist()
train_labels = df_train['label'].values.tolist()
test_tweets = df_test['text'].astype(str).values.tolist()
test_labels = df_test['label'].values.tolist()

# train_tweets = [*train_tweets, *mut101["tweet"].astype(str).values.tolist()]
# train_labels = [*train_labels,*mut101["sarcastic"].values.tolist()]

model_name = 'detecting-sarcasm'
task='sentiment'
MODEL = 'xlnet-base-cased'

tokenizer = XLNetTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True,return_tensors = 'pt')
test_encodings = tokenizer(test_tweets,truncation=True, padding=True,return_tensors = 'pt')

train_dataset = SarcasmDataset(train_encodings, y_train)
test_dataset = SarcasmDataset(test_encodings, y_test)

training_args = TrainingArguments(output_dir='./res',
                                  num_train_epochs=3,
                                  per_device_train_batch_size=32,
                                  warmup_steps=500,
                                  weight_decay=0.01,
                                  logging_dir='./logs4')

model = XLNetForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset = test_dataset,
    compute_metrics = compute_metrics,
)
trainer.train()

preds = trainer.predict(test_dataset)
preds = np.argmax(preds.predictions[:, 0:2], axis=-1)
print(classification_report(y_test, preds))
print(f1_score(y_test, preds))

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['logits_proj.bias', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss


              precision    recall  f1-score   support

           0       0.66      0.77      0.71        78
           1       0.78      0.68      0.73        96

    accuracy                           0.72       174
   macro avg       0.72      0.72      0.72       174
weighted avg       0.73      0.72      0.72       174

0.7262569832402234


In [20]:
!pip install sentencepiece

In [25]:
import sentencepiece

**DeBERTa**

In [22]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import torch
from torch import nn
from torch.utils.data import Dataset
from transformers import Trainer,TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import AutoTokenizer
from sklearn.metrics import accuracy_score, f1_score


class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get('logits')
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor([0.1, 0.3]))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


train_tweets = df_train['text'].astype(str).values.tolist()
train_labels = df_train['label'].values.tolist()
test_tweets = df_test['text'].astype(str).values.tolist()
test_labels = df_test['label'].values.tolist()

model_name = 'detecting-Sarcasm'

task='sentiment'
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL, num_labels=2, loss_function_params={"weight": [0.75, 0.25]})

train_encodings = tokenizer(train_tweets, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_tweets, truncation=True, padding=True, max_length=512)

train_dataset = SarcasmDataset(train_encodings, train_labels)
test_dataset = SarcasmDataset(test_encodings, test_labels)

training_args = TrainingArguments(
    output_dir='./res', evaluation_strategy="steps", num_train_epochs=3, per_device_train_batch_size=16,
    per_device_eval_batch_size=64, warmup_steps=500, weight_decay=0.01,logging_dir='./logs4',
    load_best_model_at_end=True,
)

model = AutoModelForSequenceClassification.from_pretrained(MODEL)

trainer = Trainer(
    model=model, args=training_args, train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)

trainer.train()

trainer.evaluate()

config.json:   0%|          | 0.00/841 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

Step,Training Loss,Validation Loss


{'eval_loss': 0.6174172759056091,
 'eval_accuracy': 0.6724137931034483,
 'eval_f1_score': 0.7046632124352332,
 'eval_runtime': 1.3317,
 'eval_samples_per_second': 130.661,
 'eval_steps_per_second': 2.253,
 'epoch': 3.0}

In [24]:
test_tweets = df_test['text'].values.tolist()
test_labels = df_test['label'].values.tolist()
test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

test_dataset = SarcasmDataset(test_encodings, test_labels)

preds = trainer.predict(test_dataset=test_dataset)

probs = torch.from_numpy(preds[0]).softmax(1)

predictions = probs.numpy()

results = np.argmax(predictions,axis=1)
print(classification_report(test_labels, results))

              precision    recall  f1-score   support

           0       0.64      0.63      0.63        78
           1       0.70      0.71      0.70        96

    accuracy                           0.67       174
   macro avg       0.67      0.67      0.67       174
weighted avg       0.67      0.67      0.67       174



**Ensemble**

In [25]:
train_tweets = df_train['text'].astype(str).values.tolist()
train_labels = df_train['label'].values.tolist()
test_tweets = df_test['text'].astype(str).values.tolist()
test_labels = df_test['label'].values.tolist()

model_dict = {'cardiffnlp/twitter-roberta-base-sentiment':[AutoTokenizer, AutoModelForSequenceClassification],
              'xlnet-base-cased':[XLNetTokenizer, XLNetForSequenceClassification],
              'bert-base-uncased':[BertTokenizer, BertForSequenceClassification]}

hard_predictions = []
soft_predictions = []

for key, val in model_dict.items():
  print(key)
  tokenizer = val[0].from_pretrained(key, num_labels=2,
                                          loss_function_params={"weight": [0.75, 0.25]})


  train_encodings = tokenizer(train_tweets, padding=True, truncation=True, max_length=512)
  test_encodings = tokenizer(test_tweets, padding=True, truncation=True, max_length=512)

  train_dataset = SarcasmDataset(train_encodings, train_labels)
  test_dataset = SarcasmDataset(test_encodings, test_labels)

  if key=='xlnet-base-cased':
    training_args = TrainingArguments(output_dir='./res',
                                      num_train_epochs=3,
                                      per_device_train_batch_size=32,
                                      warmup_steps=500,
                                      weight_decay=0.01,
                                      logging_dir='./logs4')
  elif key=='cardiffnlp/twitter-roberta-base-sentiment':
    training_args = TrainingArguments(output_dir='./res',
                                      evaluation_strategy="steps",
                                      num_train_epochs=3,
                                      per_device_train_batch_size=32,
                                      per_device_eval_batch_size=64,
                                      warmup_steps=500,
                                      weight_decay=0.01,
                                      logging_dir='./logs4',
                                      load_best_model_at_end=True)
  else:
    training_args = TrainingArguments(output_dir="output",
                                      evaluation_strategy="steps",
                                      eval_steps=500,
                                      per_device_train_batch_size=8,
                                      per_device_eval_batch_size=8,
                                      num_train_epochs=3,
                                      seed=0,
                                      load_best_model_at_end=True)


  if key=='bert-base-uncased':
    model = val[1].from_pretrained(key,num_labels=2)
  else:
    model = val[1].from_pretrained(key)

  trainer = Trainer(
      model=model, args=training_args,
      train_dataset=train_dataset,
      eval_dataset=test_dataset,
      compute_metrics=compute_metrics,
  )

  trainer.train()

  preds = trainer.predict(test_dataset=test_dataset)

  if key=='xlnet-base-cased':
    preds = trainer.predict(test_dataset)
    preds = preds.predictions[:, 0:2]
    results = np.argmax(preds, axis=-1)
  else:
    probs = torch.from_numpy(preds[0]).softmax(1)
    preds = probs.numpy()
    results = np.argmax(preds,axis=1)

  print(classification_report(test_labels, results))

  hard_predictions.append(results)
  soft_predictions.append(preds)

cardiffnlp/twitter-roberta-base-sentiment


special_tokens_map.json:   0%|          | 0.00/150 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/499M [00:00<?, ?B/s]

Step,Training Loss,Validation Loss


              precision    recall  f1-score   support

           0       0.50      0.65      0.56        78
           1       0.62      0.46      0.53        96

    accuracy                           0.55       174
   macro avg       0.56      0.56      0.55       174
weighted avg       0.56      0.55      0.54       174

xlnet-base-cased


...

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


              precision    recall  f1-score   support

           0       0.77      0.96      0.85        78
           1       0.96      0.76      0.85        96

    accuracy                           0.85       174
   macro avg       0.86      0.86      0.85       174
weighted avg       0.87      0.85      0.85       174



In [26]:
binary_soft_predictions = (soft_predictions[0][:,:-1]+soft_predictions[1]+soft_predictions[2])/3
binary_hard_predictions = np.array(np.mean(np.array(hard_predictions), axis=0) > 0.5).astype(int)

print(classification_report(test_labels, np.argmax(binary_soft_predictions, axis=1)))
print(classification_report(test_labels, binary_hard_predictions))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83        78
           1       0.91      0.77      0.84        96

    accuracy                           0.83       174
   macro avg       0.84      0.84      0.83       174
weighted avg       0.85      0.83      0.83       174

              precision    recall  f1-score   support

           0       0.68      0.82      0.74        78
           1       0.82      0.69      0.75        96

    accuracy                           0.75       174
   macro avg       0.75      0.75      0.75       174
weighted avg       0.76      0.75      0.75       174

